# Inference with a pretrained model

In [ ]:
# installation

In [2]:
# constances: 
model_path_features = '/mlodata1/hokarami/fari/tracking/results/scaled_images/results_features/_100KB_/2023-12-25_12:21:17'
model_path_fourier_features = '/mlodata1/hokarami/fari/tracking/results/scaled_images/results_fourier_10_and_f10_locality_False/_100KB_/2023-12-19_12:47:30'
model_path_fourier = '/mlodata1/hokarami/fari/tracking/results/scaled_images/results_fourier_10_and_locality_False/_100KB_/2023-12-25_14:52:09'

In [1]:
from typing import Optional
from bread.data import *
from bread.algo.tracking import AssignmentDataset, GNNTracker, AssignmentClassifier, GraphLoader, accuracy_assignment, f1_assignment

from glob import glob
from pathlib import Path
import datetime, json, argparse
import torch, os
from pprint import pprint
from sklearn.metrics import confusion_matrix
import pandas as pd

from skorch.dataset import ValidSplit
from skorch.callbacks import LRScheduler, Checkpoint, ProgressBar, EpochScoring, WandbLogger, EarlyStopping

from trainutils import SaveHyperParams, seed

from importlib import import_module

import tqdm
import re

Matplotlib is building the font cache; this may take a moment.


## Read files and convert to a format that can be used by the model if needed
our algorithm only need a mask file with .h5 format, which can be generated by the following code form the .tif files. 
if you run the segmentation with YeaZ, you can skip the conversion step since the YeaZ output is already in .h5 format.
Here, fill the path to mask file with your mask file, then run the code. It will check the format of the mask file and convert it to the format that can be used by the model if needed, then read the file.


In [ ]:
mask_file = ''
if mask_file == '':
    print('Please provide a mask file')
    exit()
# check if mask file ends with .h5
elif mask_file[-4:] != '.tif':
    print('Mask file should end with .tif')
    # TODO: Convert mask file to .h5
    exit()
# chekc if mask_file is a directory
elif os.path.isdir(mask_file):
    print('Mask file should be a file not a directory')
    # TODO: stack and convert mask file to .h5
    exit()
else:
    mask_file = Path(mask_file)
    


## Make a cofig file to determine the parameters
Here, fill the configs needed to run the model in the dictionary bellow. We provide cell_graphs config, ass_graphs config, and testing configs. These configs will override the default configs of our method.

## Making cellgraphs and assgraphs

In [7]:
!python -m bread.algo.tracking.build_cellgraphs --input_segmentation "/mlodata1/hokarami/fari/bread/data/fission/masks/240102_30C_fig_SX387_01_segmentation.h5" --scale_time 1 --output_folder '/mlodata1/hokarami/fari/tracking/generated_data/scaled_images/fissions/cell_graphs_fourier_10_f10_locality_False' --nn_threshold 60 --fov 'FOV0' --frame_max 6 --frame_min 0

build cell graph args:  {'cell_f_list': ['fourier', 10, False, ['area', 'r_equiv', 'r_maj', 'r_min', 'angel', 'ecc', 'maj_x', 'maj_y', 'min_x', 'min_y']], 'edge_f_list': ['cmtocm_x', 'cmtocm_y', 'cmtocm_len', 'cmtocm_angle', 'contour_dist'], 'scale_time': 1, 'nn_threshold': 60, 'output_folder': '/mlodata1/hokarami/fari/tracking/generated_data/scaled_images/fissions/cell_graphs_fourier_10_f10_locality_False', 'input_segmentations': ['/mlodata1/hokarami/fari/bread/data/fission/masks/240102_30C_fig_SX387_01_segmentation.h5']}
segmentation: 100%|███████████████████████████████| 1/1 [00:22<00:00, 22.98s/it]


In [14]:
!python -m bread.algo.tracking.build_assgraphs --cellgraphs_dir '/mlodata1/hokarami/fari/tracking/generated_data/scaled_images/fissions/cell_graphs_fourier_10_f10_locality_False/colony005_segmentation' --output_folder "/mlodata1/hokarami/fari/tracking/generated_data/scaled_images/ass_graphs_fourier_10_f10_locality_False" --framediff_min 1 --framediff_max 2 --t1_max 5 --t1_min 1

build assignment graph with args:  {'cellgraph_dirs': ['/mlodata1/hokarami/fari/tracking/generated_data/scaled_images/fissions/cell_graphs_fourier_10_f10_locality_False'], 'output_folder': '/mlodata1/hokarami/fari/tracking/generated_data/scaled_images/ass_graphs_fourier_10_f10_locality_False', 'framediff_min': 1, 'framediff_max': 2, 't1_max': 5, 't1_min': 1}
cellgraph:   0%|                                          | 0/1 [00:00<?, ?it/s]
t1: 0it [00:00, ?it/s]
cellgraph: 100%|█████████████████████████████████| 1/1 [00:00<00:00, 123.49it/s]


## Examine the graph neural network on custom data

In [18]:
test_config = {
    "assgraphs": "/mlodata1/hokarami/fari/tracking/generated_data/scaled_images/fissions/ass_graphs_fourier_10_f10_locality_False", # a folder containing assgraphs
    "result_dir": "/mlodata1/hokarami/fari/tracking/results/scaled_images/fissions/results_fourier_10_and_f10_locality_False", # path to a folder
    "dataset": "all", # can use a regex to only run inference on parts of files in the ass_graphts folder
    "filter_file_mb": 30, # filter file sizes exceeding `filter_file_mb` MiB (usefull when running on GPU, so you don't run out of memory)
}

In [19]:
## Inference
data = test_config['dataset']
assgraphs = test_config['assgraphs']
result_dir = test_config['result_dir']
model = model_path_fourier_features
!python test_single_model.py --data $data --assgraphs $assgraphs --resultdir $result_dir --model $model


device : cpu
Traceback (most recent call last):
  File "/mlodata1/hokarami/fari/tracking/tracking_scripts/test_single_model.py", line 142, in <module>
    regex = dataset_regex[args.data]
KeyError: 'all'
